## Hierarchical Representation for Efficient Architecture search
* 참고 링크 : https://openreview.net/pdf?id=BJQRKzbA-


![img1](../images/hier.png)

![image.png](../images/hier2.png)

* operation_dict, adjacency matrix G 구현 -> 이중 리스트로 구현
* models.py의 Node_op 부분에 추가 파라미터를 주어서 변경한다

####  Operation Dictionary (operation set)


구현 목록
* operation dictionary global  생성 
* 키 값에 해당하는 함수 만들 리턴 -> 함수마다 파라미터 값이 다르기 때문에 이 부분 주의해야 (conv, maxpool)
* Triplet unit 에 parameter 추가해서 적용
* 임의의 텐서 만들어서 넣어봐서 사이즈 변화를 관찰하자

In [1]:
import os
import sys

sys.path.insert(0,'../')

import torch
import torch.nn as nn
import numpy as np

from utils_kyy import models

print(torch.__version__)
torch.cuda.current_device()

1.0.1


0

0 : 1 x 1 conv
1 : 3 x 3  depthwise conv
2 : 3 X 3 seperable conv
3 : 3 x 3 depthwise seperable conv
3 : max pooling
4 : average pooling
5 : Identity

In [12]:
import torch
import torch.nn as nn

import math

from utils_kyy.utils_graph import load_graph, get_graph_info

class depthwise_separable_conv_3x3(nn.Module):
    def __init__(self, nin, nout, stride):
        # input node 일때, stride = 1; => size 유지
        # input node 아닐 대, stride = 2; =>  (x-1)/2 + 1
        super(depthwise_separable_conv_3x3, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin, kernel_size=3, stride=stride, padding=1, groups=nin)
        self.pointwise = nn.Conv2d(nin, nout, kernel_size=1)  # default: stride=1, padding=0, dilation=1, groups=1, bias=True

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

class conv2d_3x3(nn.Module):
    def __init__(self, nin, nout, stride):
        super(conv2d_3x3, self).__init__()
        self.conv = nn.Conv2d(nin, nout, kernel_size=3, stride=stride, padding=1)

    def forward(self, x):
        out = self.conv(x)
        return out

class depthwise_conv_3x3(nn.Module):
    def __init__(self, nin, nout, stride):
        # input node 일때, stride = 1; => size 유지
        # input node 아닐 대, stride = 2; =>  (x-1)/2 + 1
        super(depthwise_conv_3x3, self).__init__()
        self.depthwise = nn.Conv2d(nin, nout, kernel_size=3, stride=stride, padding=1, groups=nin)

    def forward(self, x):
        out = self.depthwise(x)
        return out

class separable_conv_3x3(nn.Module):
    def __init__(self, nin, nout, stride):
        # input node 일때, stride = 1; => size 유지
        # input node 아닐 대, stride = 2; =>  (x-1)/2 + 1
        super(separable_conv_3x3, self).__init__()
        self.pointwise = nn.Conv2d(nin, nout, kernel_size=3, stride=stride)  # default: stride=1, padding=0, dilation=1, groups=1, bias=True

    def forward(self, x):
        out = self.pointwise(x)
        return out

class maxpool2d_3x3(nn.Module):
    def __init__(self, nin, nout, stride):
        super(maxpool2d_3x3, self).__init__()
        self.conv = nn.MaxPool2d(kernel_size=3, stride=stride, padding=1)

    def forward(self, x):
        out = self.conv(x)
        return out

class avgpool2d_3x3(nn.Module):
    def __init__(self, nin, nout, stride):
        super(avgpool2d_3x3, self).__init__()
        self.conv = nn.AvgPool2d(kernel_size=3, stride=stride, padding=1)

    def forward(self, x):
        out = self.conv(x)
        return out
    
class identity(nn.Module):
    def __init__(self, nin, nout, stride):
        # input node 일때, stride = 1; => size 유지
        # input node 아닐 대, stride = 2; =>  (x-1)/2 + 1
        super(identity, self).__init__()

    def forward(self, x):
        out = x
        return out
 
    
class Triplet_unit(nn.Module):
    def __init__(self, inplanes, outplanes, stride=1):
        super(Triplet_unit, self).__init__()
        self.relu = nn.ReLU()
        self.conv = depthwise_separable_conv_3x3(inplanes, outplanes, stride)
        self.bn = nn.BatchNorm2d(outplanes)

    def forward(self, x):
        out = self.relu(x)
        out = self.conv(out)
        out = self.bn(out)
        return out

## new triplet_unint
class Triplet_unit(nn.Module):
    def __init__(self, inplanes, outplanes, operation=1, stride=1):
        super(Triplet_unit, self).__init__()
        self.relu = nn.ReLU()
        self.operation = 1
    
        self.conv = depthwise_separable_conv_3x3(inplanes, outplanes, stride)
        self.bn = nn.BatchNorm2d(outplanes)

    def forward(self, x):
        out = self.relu(x)
        out = self.conv(out)
        out = self.bn(out)
        return out

    
class Node_OP(nn.Module):
    def __init__(self, Node, inplanes, outplanes):
        super(Node_OP, self).__init__()
        self.is_input_node = Node.type == 0
        self.input_nums = len(Node.inputs)    # 해당 Node에 input으로 연결된 노드의 개수

        # input 개수가 1보다 크면, 여러 input을 합쳐야함.
        if self.input_nums > 1:
            self.mean_weight = nn.Parameter(torch.ones(self.input_nums))  # type: torch.nn.parameter.Parameter
            self.sigmoid = nn.Sigmoid()

        if self.is_input_node:
            self.conv = Triplet_unit(inplanes, outplanes, stride=2)   # Triplet_unit = relu, conv, bn
        else:
            self.conv = Triplet_unit(outplanes, outplanes, stride=1)

    # [참고] nn.Sigmoid()(torch.ones(1)) = 0.7311
    # seoungwonpark source 에서는 torch.zeros()로 들어감. => 0.5
    def forward(self, *input):
        if self.input_nums > 1:
            out = self.sigmoid(self.mean_weight[0]) * input[0]
            for i in range(1, self.input_nums):
                out = out + self.sigmoid(self.mean_weight[i]) * input[i]
        else:
            out = input[0]
        out = self.conv(out)
        return out

    

class StageBlock(nn.Module):
    def __init__(self, graph, inplanes, outplanes):
        super(StageBlock, self).__init__()
        # graph를 input으로 받아서, Node_OP class. 즉, neural network graph로 전환함.
        self.nodes, self.input_nodes, self.output_nodes = get_graph_info(graph)
        self.nodeop  = nn.ModuleList()    # Holds submodules in a list.
        for node in self.nodes:
            # 각각의 node들을 Node_OP class로 만들어준 뒤, nn.ModuleList()인 self.nodeop에 append 해주기
            self.nodeop.append(Node_OP(node, inplanes, outplanes))

    def forward(self, x):
        results = {}
        # input
        for id in self.input_nodes:
            results[id] = self.nodeop[id](x)  # input x를 먼저 graph's input node에 각각 넣어줌.

        # graph 중간 계산
        for id, node in enumerate(self.nodes):
            # 각각의 노드 id에 대해
            if id not in self.input_nodes:
                # graph's input node가 아니라면, 그래프 내에서 해당 노드의 인풋들인 node.inputs의 output인 results[_id]
                #    => 그 결과를 results[id]에 저장.
                # self.nodeop[id]는 해당 id의 Node_OP. 즉, input들을 받아서 forward(모아서, conv 태우기)하는 것.
                # 따라서, input으로 넣을 때 unpack 함.
                # id 작은 노드부터 result를 차근차근 계산하면서, id를 올라감.
                results[id] = self.nodeop[id](*[results[_id] for _id in node.inputs])

        result = results[self.output_nodes[0]]
        # output
        # graph's output_nodes의 output 들을 평균내기
        for idx, id in enumerate(self.output_nodes):
            if idx > 0:
                result = result + results[id]
        result = result / len(self.output_nodes)
        return result
   
 
# Node_OP -> StageBlock class 정의해놓고,
# conv2, conv3, conv4에 각각 random graph 생성해서 모듈로 추가함
# e.g.
#  graphs = EasyDict({'stage_1': stage_1_graph,
#                     'stage_2': stage_2_graph,
#                     'stage_3': stage_3_graph
#  })   # stage_1_graph = 해당 graph 파일의 path
# channels = 109

class RWNN(nn.Module):
    def __init__(self, net_type, graphs, channels, num_classes=1000, input_channel=3):
        super(RWNN, self).__init__()

        self.input_channel = input_channel
        # 논문에서도 conv1 쪽은 예외적으로 Conv-BN 이라고 언급함. (나머지에서는 Conv-ReLU-BN 을 conv 로 표기) 
#         self.conv1 = depthwise_separable_conv_3x3(input_channel, channels // 2, 2)    # nin, nout, stride
        self.conv1 = depthwise_separable_conv_3x3(input_channel, channels // 2, 1)    # nin, nout, stride        
        self.bn1 = nn.BatchNorm2d(channels // 2)
        
        # 채널수 변화도, 논문에서처럼 conv2: C, conv3: 2C, conv4: 4C, conv5: 8C
        if net_type == 'small':
#             self.conv2 = Triplet_unit(channels // 2, channels, 2)    # inplanes, outplanes, stride=2
            self.conv2 = Triplet_unit(channels // 2, channels, 1)    # inplanes, outplanes, stride=1            

#             self.conv3 = StageBlock(graphs.stage_1, channels // 2, channels)
            self.conv3 = StageBlock(graphs.stage_1, channels, channels)        
 
            self.conv4 = StageBlock(graphs.stage_2, channels, channels * 2)   

            self.conv5 = StageBlock(graphs.stage_3, channels * 2, channels * 4)

            self.relu = nn.ReLU()
            
            self.bn2 = nn.BatchNorm2d(channels * 4)
            self.avgpool = nn.AvgPool2d(4, stride=1)  # 마지막은 global average pooling
            self.fc = nn.Linear(channels * 4, num_classes)

#         if net_type == 'small':
#             self.conv2 = Triplet_unit(channels // 2, channels, 2)    # inplanes, outplanes, stride=2

#             self.conv3 = StageBlock(graphs.stage_1, channels, channels)
 
#             self.conv4 = StageBlock(graphs.stage_2, channels, channels *2)   

#             self.conv5 = StageBlock(graphs.stage_3, channels * 2, channels * 4)

#             self.relu = nn.ReLU()
# #             self.conv = nn.Conv2d(channels * 4, 1280, kernel_size=1)   # 마지막에 1x1 conv, 1280-d
# #             self.bn2 = nn.BatchNorm2d(1280)
        
        #######################################
        # 원 코드에서 regular 부분 지움
        #######################################
#         self.avgpool = nn.AvgPool2d(7, stride=1)  # 마지막은 global average pooling
#         self.fc = nn.Linear(1280, num_classes)
    
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
        
        
    def forward(self, x):
#         print("==========Data Size", x.size())
        x = self.conv1(x)
        x = self.bn1(x)
#         print("==========Conv1 Size", x.size())  # => (-, 54, 16, 16)

        x = self.conv2(x)
#         print("==========Conv2  Size", x.size())  # => (-, 109, 8, 8)

        x = self.conv3(x)
#         print("==========Conv3  Size", x.size())  # => (-, 109, 8, 8)

        x = self.conv4(x)
#         print("==========Conv4  Size", x.size())  # => (-, 218, 4, 4)

        x = self.conv5(x)
#         print("==========Conv5  Size", x.size())  # => (-, 436, 2, 2)

#         x = self.relu(x)  # => (-, 436, 1, 1)
#         x = self.conv(x)
#         print("==========Conv6  Size", x.size())  # => (-, 1280, 1, 1)

        x = self.bn2(x)
        x = self.relu(x)

#         print("==========before avgpool  Size", x.size())   ## [수정] CIFAR-10 에서는 여기까지오면 (-, 1280, 7, 7) 이 아니라 (-, ,1280, 1, 1)
        x = self.avgpool(x)
#         print("==========after avgpool  Size", x.size())
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
    

###################################################    
# #print("==========Data Size", x.size())
# x = self.conv1(x)
# x = self.bn1(x)

# x = self.conv2(x)
# x = self.conv3(x)
# x = self.conv4(x)
# x = self.conv5(x)

# x = self.relu(x)
# x = self.conv(x)

# x = self.bn2(x)
# x = self.relu(x)
# x = self.avgpool(x)

# x = x.view(x.size(0), -1)
# x = self.fc(x)


In [13]:

def operation_dictionary():
    temp_dict = {}
    temp_dict[0] = conv2d_3x3 # nin, nout, kernel_size=3, stride=stride, padding=
    temp_dict[1] = depthwise_conv_3x3 # nin, nout ,  stride
    temp_dict[2] = separable_conv_3x3
    temp_dict[3] = depthwise_separable_conv_3x3
    temp_dict[4] = maxpool2d_3x3 # parameter Kernel_size , stride, padding
    temp_dict[5] = avgpool2d_3x3
    temp_dict[6] = identity
    
    return temp_dict

In [14]:
oper_dict = operation_dictionary()

In [15]:
oper_dict

{0: __main__.conv2d_3x3,
 1: __main__.depthwise_conv_3x3,
 2: __main__.separable_conv_3x3,
 3: __main__.depthwise_separable_conv_3x3,
 4: __main__.maxpool2d_3x3,
 5: __main__.avgpool2d_3x3,
 6: __main__.identity}

* Operation 검증

In [16]:
i = np.array([[[[1,0,0],
             [0,0,0],
             [0,0,0]],
            [[0,0,0],
            [0,0,0],
            [0,0,0]],
            [[0,0,0],
            [0,0,0],
            [0,0,0]]]], dtype='float32')

In [17]:
i.shape

(1, 3, 3, 3)

In [18]:
idx = 0
operation  = oper_dict[idx]
# (Batch size , Cin, H, W)
tesnor_ex = torch.from_numpy(i)

tesnor_ex.size()

t = operation(3,3,stride=1)

print(operation)
print('='*30 + '\n Result ' )
print(t.forward(tesnor_ex))
print('='*30 + '\n Parameter ' )
print(list(t.parameters()))
print('='*30 + '\n Tensor ' )
print(tesnor_ex)

<class '__main__.conv2d_3x3'>
 Result 
tensor([[[[-0.2049,  0.1401, -0.0421],
          [-0.0809,  0.0158, -0.0421],
          [-0.0421, -0.0421, -0.0421]],

         [[ 0.0854,  0.2330,  0.0586],
          [ 0.2352,  0.1584,  0.0586],
          [ 0.0586,  0.0586,  0.0586]],

         [[ 0.0670,  0.1142,  0.1676],
          [ 0.1407,  0.3544,  0.1676],
          [ 0.1676,  0.1676,  0.1676]]]], grad_fn=<ThnnConv2DBackward>)
 Parameter 
[Parameter containing:
tensor([[[[ 0.0578, -0.0388, -0.0424],
          [ 0.1822, -0.1628,  0.0745],
          [ 0.1262,  0.1321, -0.0093]],

         [[-0.0481,  0.0705,  0.1737],
          [-0.0493,  0.1760, -0.1607],
          [-0.0215,  0.1035, -0.0457]],

         [[ 0.0039,  0.0318, -0.1517],
          [ 0.0531, -0.1897, -0.0113],
          [ 0.1197, -0.1011,  0.0810]]],


        [[[ 0.0998,  0.1767, -0.0646],
          [ 0.1744,  0.0269, -0.1647],
          [ 0.0276, -0.0303,  0.0286]],

         [[ 0.0006, -0.1863, -0.1442],
          [-0.1001, -

In [9]:
idx = 2
operation  = oper_dict[idx]
# (Batch size , Cin, H, W)
tesnor_ex = torch.from_numpy(i)
#tesnor_ex = torch.zeros([1,3,3,3])
tesnor_ex.size()

t = operation(3,3,stride=1)

print(operation)
print('='*30 + '\n Result ' )
print(t.forward(tesnor_ex))
print('='*30 + '\n Parameter ' )
print(list(t.parameters()))
print('='*30 + '\n Tensor ' )
print(tesnor_ex)




<class '__main__.separable_conv_3x3'>
 Result 
tensor([[[[ 0.3604,  0.4085,  0.4085],
          [ 0.4085,  0.4085,  0.4085],
          [ 0.4085,  0.4085,  0.4085]],

         [[ 0.0270, -0.2240, -0.2240],
          [-0.2240, -0.2240, -0.2240],
          [-0.2240, -0.2240, -0.2240]],

         [[-0.9502, -0.4955, -0.4955],
          [-0.4955, -0.4955, -0.4955],
          [-0.4955, -0.4955, -0.4955]]]], grad_fn=<ThnnConv2DBackward>)
 Parameter 
[Parameter containing:
tensor([[[[-0.0481]],

         [[-0.4679]],

         [[-0.4895]]],


        [[[ 0.2510]],

         [[ 0.3011]],

         [[ 0.4044]]],


        [[[-0.4547]],

         [[-0.4290]],

         [[-0.2246]]]], requires_grad=True), Parameter containing:
tensor([ 0.4085, -0.2240, -0.4955], requires_grad=True)]
 Tensor 
tensor([[[[1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0.

In [21]:
idx = 4
operation  = oper_dict[idx]
# (Batch size , Cin, H, W)
tesnor_ex = torch.from_numpy(i)
#tesnor_ex = torch.zeros([1,3,3,3])
tesnor_ex.size()

t = operation(3,3,1)

print(operation)
print('='*30 + '\n Result ' )
print(t.forward(tesnor_ex), t.forward(tesnor_ex).size())
print('='*30 + '\n Parameter ' )
print(list(t.parameters()))
print('='*30 + '\n Tensor ' )
print(tesnor_ex)




<class '__main__.maxpool2d_3x3'>
 Result 
tensor([[[[1., 1., 0.],
          [1., 1., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]]) torch.Size([1, 3, 3, 3])
 Parameter 
[]
 Tensor 
tensor([[[[1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]])


* TO DO 
Maxpooling -> 객체화
identity 까지 검증하고  -> triplet 적용 -> 바로 실험 오늘까지

In [22]:
idx = 5
operation  = oper_dict[idx]
# (Batch size , Cin, H, W)
tesnor_ex = torch.from_numpy(i)
#tesnor_ex = torch.zeros([1,3,3,3])
tesnor_ex.size()

t = operation(3,3,1)

print(operation)
print('='*30 + '\n Result ' )
print(t.forward(tesnor_ex), t.forward(tesnor_ex).size())
print('='*30 + '\n Parameter ' )
print(list(t.parameters()))
print('='*30 + '\n Tensor ' )
print(tesnor_ex)




<class '__main__.avgpool2d_3x3'>
 Result 
tensor([[[[0.1111, 0.1111, 0.0000],
          [0.1111, 0.1111, 0.0000],
          [0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]]]]) torch.Size([1, 3, 3, 3])
 Parameter 
[]
 Tensor 
tensor([[[[1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]])


In [23]:
idx = 6
operation  = oper_dict[idx]
# (Batch size , Cin, H, W)
tesnor_ex = torch.from_numpy(i)
#tesnor_ex = torch.zeros([1,3,3,3])
tesnor_ex.size()

t = operation(3,3,1)

print(operation)
print('='*30 + '\n Result ' )
print(t.forward(tesnor_ex), t.forward(tesnor_ex).size())
print('='*30 + '\n Parameter ' )
print(list(t.parameters()))
print('='*30 + '\n Tensor ' )
print(tesnor_ex)




<class '__main__.identity'>
 Result 
tensor([[[[1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]]) torch.Size([1, 3, 3, 3])
 Parameter 
[]
 Tensor 
tensor([[[[1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]])


* Operation DIctionary 생성 완료